In [13]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype
from datetime import datetime
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

from surprise import Dataset
from surprise.reader import Reader
from surprise.prediction_algorithms.matrix_factorization import SVD as FunkSVD
from funk_svd.dataset import fetch_ml_ratings
from funk_svd import SVD
from sklearn.metrics import mean_absolute_error

In [8]:
movies= pd.read_csv("Data/movies.csv")

In [4]:
ratings_sample = pd.read_csv("Data/ratings_sample.csv")

In [5]:
ratings_sample.drop("Unnamed: 0",axis=1,inplace=True)

In [6]:
ratings_sample

,userId,movieId,rating,liked
0,154008,356,3.5,1
1,19168,45186,3.5,1
2,108927,786,4.5,1
3,190876,2193,4.0,1
4,243682,54999,4.0,1
...,...,...,...,...
855257,174944,2001,2.0,0
855258,189774,193,2.0,0
855259,97970,55442,3.0,0
855260,125224,329,3.0,0


In [9]:
movies

,Unnamed: 0,movieId,imdb_id,original_language,original_title,overview,popularity,release_date,runtime,title,...,Family.6,Foreign.6,Music.6,Romance.6,Science Fiction.6,Thriller.6,Western.6,Comedy.7,Family.7,Thriller.7
0,0,862.000000,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30 00:00:00,81.000000,Toy Story,...,0,0,0,0,0,0,0,0,0,0
1,1,8844.000000,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15 00:00:00,104.000000,Jumanji,...,0,0,0,0,0,0,0,0,0,0
2,4,11862.000000,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,1995-02-10 00:00:00,106.000000,Father of the Bride Part II,...,0,0,0,0,0,0,0,0,0,0
3,5,949.000000,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.924927,1995-12-15 00:00:00,170.000000,Heat,...,0,0,0,0,0,0,0,0,0,0
4,6,11860.000000,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,6.677277,1995-12-15 00:00:00,127.000000,Sabrina,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6278,45343,248705.000000,tt2441982,fr,Les Visiteurs: La Révolution,"Stuck in the corridors of time, Godefroy de Mo...",7.294920,2016-03-23 00:00:00,110.000000,The Visitors: Bastille Day,...,0,0,0,0,0,0,0,0,0,0
6279,45377,94.115707,94.1157070494813,94.1157070494813,94.1157070494813,94.1157070494813,94.115707,1970-01-01 00:00:00.000000094,94.115707,94.1157070494813,...,0,0,0,0,0,0,0,0,0,0
6280,45410,94.115707,94.1157070494813,94.1157070494813,94.1157070494813,94.1157070494813,94.115707,1970-01-01 00:00:00.000000094,94.115707,94.1157070494813,...,0,0,0,0,0,0,0,0,0,0
6281,45437,455661.000000,tt6969946,en,In a Heartbeat,A closeted boy runs the risk of being outed by...,20.821780,2017-06-01 00:00:00,4.000000,In a Heartbeat,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# data = pd.read_csv("merged_data.csv")

## use similarity matrix


In [6]:
# Create a user-item matrix



<ipython-input-6-3fe9b7aec15b>:2: PerformanceWarning: The following operation may generate 4668692358 cells in the resulting pandas object.
  user_item_matrix = ratings_sample.groupby(['userId', 'movieId'])['rating'].mean().unstack(fill_value=0)


In [10]:
# Identify unique users and items
unique_users = ratings_sample['userId'].unique()
unique_items = ratings_sample['movieId'].unique()

# Create an empty user-item matrix
user_item_matrix = np.zeros((len(unique_users), len(unique_items)))

# Populate the user-item matrix
for index, row in ratings_sample.iterrows():
    user_id = row['userId']
    item_id = row['movieId']
    rating = row['rating']

    user_index = np.where(unique_users == user_id)[0][0]
    item_index = np.where(unique_items == item_id)[0][0]

    user_item_matrix[user_index, item_index] = rating

# Print the user-item matrix
print(user_item_matrix)

[[3.5 0.  0.  ... 0.  0.  0. ]
 [0.  3.5 0.  ... 0.  0.  0. ]
 [0.  0.  4.5 ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]]


In [11]:

df = pd.DataFrame(data=user_item_matrix, index=[unique_users],
                  columns=[unique_items])

In [12]:
df.to_csv("user_item.csv")

## Collaborative filtering

In [14]:
#using Funk SVD
my_dataset = Dataset.load_from_df(df, ratings_sample(rating_scale=(1, 5)))
my_train_dataset = my_dataset.build_full_trainset()

ValueError: too many values to unpack (expected 3)

In [117]:
df.loc[:,10]

,10
154008,0.0
19168,0.0
108927,0.0
190876,0.0
243682,0.0
...,...
10070,0.0
39471,0.0
142018,0.0
6361,0.0


In [126]:
df.loc[131060	,:]

,356,45186,786,2193,54999,4637,4403,107069,4878,1729,...,159803,71486,140196,27454,108332,76066,79994,100246,153969,59180
131060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [127]:
df.loc[154008,:]

,356,45186,786,2193,54999,4637,4403,107069,4878,1729,...,159803,71486,140196,27454,108332,76066,79994,100246,153969,59180
154008,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
#Calculate user similarities
def find_user_similarity(user_1, user_2,df):

    # Define the mask which finds all movies they rated together
    movies_1st_user_rated = df.loc[user_1, :].notna()
    movies_2nd_user_rated = df.loc[user_2, :].notna()

    movies_both_users_rated = movies_1st_user_rated & movies_2nd_user_rated

    # Sum boolean to get the counts
    number_of_movies_rated_together = movies_both_users_rated.sum()

    # Find the ratings of both users for movies they both watched
    ratings_of_user1 = df.loc[user_1, movies_both_users_rated].values.reshape(1, -1)
    ratings_of_user2 = df.loc[user_2, movies_both_users_rated].values.reshape(1, -1)

    # Finally, calculate the similarity between them
    similarity = cosine_similarity(ratings_of_user1, ratings_of_user2)[0][0]

    return similarity, number_of_movies_rated_together




In [128]:
current_user =  154008
current_movie = 356
other_user = 131060
similarities_to_user = []
ratings_given_to_movie = []

# Find only the users who rated movie 2 (rows)
df2 = df[df.iloc[:, 1].isna()].copy()

for other_user in df2.index:

    similarity, number_of_movies_rated_together = find_user_similarity(current_user,
                                                                       other_user, df)
    similarities_to_user.append(similarity)
    ratings_given_to_movie.append(df.loc[other_user, current_movie])

# Finally, let's turn these into numpy arrays so life is easier
similarities_to_user = np.array(similarities_to_user)
ratings_given_to_movie = np.array(ratings_given_to_movie)

In [129]:
predicted_rating = np.dot(ratings_given_to_movie,
                      similarities_to_user)/np.sum(similarities_to_user)



<ipython-input-129-8462a84eb715>:1: RuntimeWarning: invalid value encountered in double_scalars
  predicted_rating = np.dot(ratings_given_to_movie,


In [130]:
predicted_rating

nan

## Generate recommendations based on genre

## Recommendation based on Overview

### Funk SVD

In [ ]:
my_dataset = Dataset.load_from_df(ratings_sample, Reader(rating_scale=(1, 5)))
my_train_dataset = my_dataset.build_full_trainset()

In [ ]:
my_algorithm = FunkSVD(n_factors=10,
                       n_epochs=100,
                       lr_all=0.1,
                       biased=False,
                       verbose=0)

my_algorithm.fit(my_train_dataset)

In [ ]:
U = my_algorithm.pu
U.shape

In [ ]:
M = my_algorithm.qi.T
M.shape

In [ ]:
inner_user_id = my_train_dataset.to_inner_uid(user_1) # find the inner representation of user 1
user_profile = U[inner_user_id]
user_profile

In [ ]:
inner_movie_id = my_train_dataset.to_inner_iid(862)
movie_profile = M[:, inner_movie_id]
movie_profile

In [ ]:
expected_rating = np.dot(user_profile, movie_profile)
expected_rating